# Download Historical Data

In [1]:
import pfeed as pe

pe.__version__

'0.0.1.dev12'

## Download Historical Data from Bybit

In [12]:
pe.bybit.download_historical_data(
    products=['BTC_USDT_PERP'],
    dtypes=['raw'],  # this will be converted to `raw_tick` implicitly for Bybit
    start_date='2024-03-01',
    end_date='2024-03-02',
    num_cpus=8,  # num_cpus for Ray
    use_ray=True,
    use_minio=False,
)

Hint: 
        You can use the command "pfeed config --data-path ..." to set your data path that stores downloaded data.
        The current data path is: /Users/stephenyau/pfund.ai/pfeed/.local/data.
    


Ray is enabled

MinIO is disabled

PFeed: downloading historical data from BYBIT, start_date='2024-03-01' end_date='2024-03-02'

2024-09-03 23:53:38,020	INFO worker.py:1781 -- Started a local Ray instance.


Ray's num_cpus is set to 8


(_run_task pid=71375) Disk usage at /Users/stephenyau/pfund.ai/pfeed/.local/data: 44.8%


2024-09-03T23:53:50+0800.508 | INFO | bybit_data | loaded BYBIT data to /Users/stephenyau/pfund.ai/pfeed/.local/data/bybit/historical/minute/BTC_USDT_PERP/BTC_USDT_PERP_2024-03-01.parquet | fn:load_data ln:194
2024-09-03T23:53:50+0800.526 | WARNING | bybit_data | finished downloading BYBIT historical data to /Users/stephenyau/pfund.ai/pfeed/.local/data | fn:download_historical_data ln:189


(_run_task pid=71378) Disk usage at /Users/stephenyau/pfund.ai/pfeed/.local/data: 44.8%


```{note}
1. You can also download data in different ways:
- pe.download(data_source='bybit', ...)
- pe.BybitFeed().download(...)

2. If you do not need the raw data, you can download the resampled data directly by specifying the `dtypes` argument. e.g. `dtypes=['minute']` will download 1-minute data.
```

Now you can get your downloaded raw data locally by creating a Feed object:
> if the data has not been downloaded, it will be downloaded on the fly automatically.

In [6]:
bybit_feed = pe.BybitFeed()
raw_df = bybit_feed.get_historical_data(
    product='BTC_USDT_PERP',
    resolution='raw',
    start_date='2024-03-01',
    end_date='2024-03-02',
)
raw_df.head(3)

2024-09-03T23:35:28+0800.246 | INFO | bybit_data | loaded BYBIT BTC_USDT_PERP 2024-03-01 raw_tick data locally | fn:_get_data_from_local ln:122
2024-09-03T23:35:28+0800.762 | INFO | bybit_data | loaded BYBIT BTC_USDT_PERP 2024-03-02 raw_tick data locally | fn:_get_data_from_local ln:122


,ts,symbol,side,volume,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional
0,2024-03-01 00:00:00.097599983,BTCUSDT,1,0.003,61184.1,ZeroMinusTick,79ac9a21-0249-5985-b042-906ec7604794,1.835523e+10,0.003,183.5523
1,2024-03-01 00:00:00.098299980,BTCUSDT,1,0.078,61184.9,PlusTick,2af4e516-8ff4-5955-bb9c-38aa385b7b44,4.772422e+11,0.078,4772.4222
2,2024-03-01 00:00:00.098299980,BTCUSDT,1,0.070,61185.2,PlusTick,e4ce9cf1-c32b-5a1a-a211-1a16ea160d67,4.282964e+11,0.070,4282.9640


## Resample Data

You can resample the downloaded raw data to other resolutions by specifying the `resolution` argument, e.g. 1m or 1minute for 1-minute data.

In [11]:
minute_df = bybit_feed.get_historical_data(
    product='BTC_USDT_PERP',
    resolution='1m',
    start_date='2024-03-01',
    end_date='2024-03-02',
)
minute_df.head(3)

2024-09-03T23:53:13+0800.158 | INFO | bybit_data | No local BYBIT data found with dtype='minute', switched to find "raw_tick" data instead | fn:_get_data_from_local ln:116
2024-09-03T23:53:13+0800.159 | INFO | bybit_data | loaded BYBIT BTC_USDT_PERP 2024-03-01 raw_tick data locally | fn:_get_data_from_local ln:122


2024-09-03T23:53:13+0800.907 | INFO | bybit_data | resampled BYBIT raw tick  data to resolution=1m | fn:_transform_raw_data_to_dtype ln:49
2024-09-03T23:53:13+0800.927 | INFO | bybit_data | No local BYBIT data found with dtype='minute', switched to find "raw_tick" data instead | fn:_get_data_from_local ln:116
2024-09-03T23:53:13+0800.928 | INFO | bybit_data | loaded BYBIT BTC_USDT_PERP 2024-03-02 raw_tick data locally | fn:_get_data_from_local ln:122
2024-09-03T23:53:14+0800.595 | INFO | bybit_data | resampled BYBIT raw tick  data to resolution=1m | fn:_transform_raw_data_to_dtype ln:49


,ts,product,resolution,open,high,low,close,volume
0,2024-03-01 00:00:00,BTC_USDT_PERP,1m,61184.1,61244.5,61175.8,61244.5,159.142
1,2024-03-01 00:01:00,BTC_USDT_PERP,1m,61245.3,61276.5,61200.7,61232.2,227.242
2,2024-03-01 00:02:00,BTC_USDT_PERP,1m,61232.2,61249.0,61180.0,61184.2,91.446
